In [ ]:
!pip install flask


In [ ]:
!pip install vllm torch pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 M

In [ ]:
!ngrok config add-authtoken YOUR_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import torch
from flask import Flask, request, jsonify, Response, stream_with_context
from vllm import LLM, SamplingParams
import json
import gc
from datetime import datetime
import time

# ============================================================================
# MODEL YÜKLEME
# ============================================================================
print("=" * 80)
print("🚀 MODEL YÜKLEME BAŞLIYOR...")
print(f"⏰ Zaman: {datetime.now().strftime('%H:%M:%S')}")
print("=" * 80)

try:
    llm_planner = LLM(
        model="ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
        trust_remote_code=True,
        gpu_memory_utilization=0.90,
        max_model_len=4096,
        dtype="float16"
    )
    print("✅ Model başarıyla yüklendi!")
    print(f"🔧 GPU Kullanımı: %90")
    print(f"📏 Max Token: 4096")

    # Tokenizer'ı al (chat template için)
    tokenizer = llm_planner.get_tokenizer()
    print(f"✅ Tokenizer hazır: {tokenizer.__class__.__name__}")

except Exception as e:
    print(f"❌ Model yükleme hatası: {e}")
    raise

# ============================================================================
# SİSTEM PROMPTU
# ============================================================================
SYSTEM_PROMPT = """Sen uzman bir proje analiz asistanısın. Görüvin Jira verilerini analiz edip projenin mevcut durumu hakkında detaylı, yapıcı ve net değerlendirmeler yapmak.

GÖREV TANIMLARIN:
1. Proje Sağlığı Değerlendirmesi - Issue durumlarını analiz et
2. Risk Tespiti - Gecikmeler, bloke durumlar, atanmamış işleri belirle
3. Sprint Performansı - İlerlemeyi ve hedeflere ulaşmayı değerlendir
4. Kaynak Dağılımı - Ekip iş yükü dağılımını incele
5. Öncelik Analizi - Yüksek öncelikli görevleri kontrol et

ÇIKTI FORMATI:

PROJE ÖZET:
- Toplam görev sayısı ve durum dağılımı
- Tamamlanma yüzdesi

RİSKLER VE DİKKAT GEREKENLER:
- Kritik riskler
- Gecikmeler
- Bloke durumlar

ÖNCELİK DURUMU:
- Yüksek öncelikli görevlerin durumu
- Acil müdahale gereken konular

EKİP PERFORMANSI:
- İş yükü dağılımı
- Atanmamış görevler

ÖNERİLER:
- Kısa vadeli aksiyon önerileri
- Süreç iyileştirme önerileri

Net, anlaşılır ve Türkçe yaz. Rakamlarla destekle."""

# ============================================================================
# YARDIMCI FONKSİYONLAR
# ============================================================================
def format_chat_prompt(system_msg: str, user_msg: str) -> str:
    """Chat template ile prompt formatla"""
    try:
        # Llama chat format
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ]

        # Tokenizer'ın chat template'i varsa kullan
        if hasattr(tokenizer, 'apply_chat_template'):
            formatted = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            print("✅ Chat template uygulandı")
            return formatted
        else:
            # Manuel format (Llama style)
            formatted = f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_msg} [/INST]"
            print("✅ Manuel Llama format uygulandı")
            return formatted

    except Exception as e:
        print(f"⚠️ Chat template hatası: {e}")
        # Fallback: basit format
        return f"{system_msg}\n\n{user_msg}\n\nYanıt:"

def create_analysis_prompt(json_data: dict, project_key: str) -> str:
    """JSON verisini analiz için prompt haline getirir"""
    try:
        print(f"📝 Prompt oluşturuluyor...")
        json_str = json.dumps(json_data, indent=2, ensure_ascii=False)

        user_message = f"""Aşağıdaki {project_key} projesi verilerini analiz et ve belirlenen formatta detaylı bir rapor hazırla.

PROJE VERİLERİ:
{json_str}

Lütfen yukarıdaki verileri dikkatlice analiz edip tam ve detaylı bir rapor oluştur. Her başlık altında somut bulgular ve öneriler sun."""

        prompt = format_chat_prompt(SYSTEM_PROMPT, user_message)

        print(f"✅ Prompt hazır - Uzunluk: {len(prompt)} karakter")
        return prompt
    except Exception as e:
        error_msg = f"❌ Prompt oluşturma hatası: {str(e)}"
        print(error_msg)
        return error_msg

def generate_streaming_response(prompt: str):
    """vLLM ile streaming text üretir"""

    # DÜZELTİLMİŞ SAMPLING PARAMETRELERİ
    sampling_params = SamplingParams(
        temperature=0.7,           # Biraz daha yaratıcı
        top_p=0.95,
        top_k=50,
        max_tokens=2048,           # Daha uzun çıktı için
        repetition_penalty=1.15,   # Tekrarı azalt
        presence_penalty=0.1,      # Çeşitliliği artır
        frequency_penalty=0.1,
        stop=None,                 # Stop token'ları kaldır!
        skip_special_tokens=True   # Özel token'ları atla
    )

    try:
        print("\n" + "🔄" * 40)
        print(f"🤖 MODEL ÇALIŞIYOR...")
        print(f"⏰ Başlangıç: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
        print(f"📊 Prompt uzunluğu: {len(prompt)} karakter")
        print(f"🎚️ Temperature: {sampling_params.temperature}")
        print(f"🎯 Max tokens: {sampling_params.max_tokens}")
        print(f"🛑 Stop tokens: {sampling_params.stop}")

        # Model üretimi
        start_time = time.time()
        outputs = llm_planner.generate([prompt], sampling_params, use_tqdm=False)
        generation_time = time.time() - start_time

        full_text = outputs[0].outputs[0].text.strip()

        # Debug: finish reason kontrol et
        finish_reason = outputs[0].outputs[0].finish_reason
        print(f"\n🏁 Finish Reason: {finish_reason}")

        print(f"\n✅ MODEL ÇIKTI ÜRETTİ!")
        print(f"📏 Çıktı uzunluğu: {len(full_text)} karakter")
        print(f"📏 Token sayısı: ~{len(full_text.split())}")
        print(f"⏱️ Üretim süresi: {generation_time:.2f} saniye")
        print(f"🚀 Hız: {len(full_text)/generation_time:.1f} karakter/saniye")

        # İlk 200 karakteri göster
        preview = full_text[:200] + "..." if len(full_text) > 200 else full_text
        print(f"\n📖 Önizleme:\n{preview}")
        print("🔄" * 40 + "\n")

        # Streaming başlangıcı
        print("📡 STREAMING BAŞLIYOR...")
        stream_start = time.time()

        # Metni chunk'lar halinde stream et
        chunk_size = 8  # Her seferde 8 karakter
        for i in range(0, len(full_text), chunk_size):
            chunk = full_text[i:i+chunk_size]
            yield f"data: {json.dumps({'text': chunk, 'done': False}, ensure_ascii=False)}\n\n"

            # Her 200 karakterde bir durum raporu
            if i % 200 == 0 and i > 0:
                progress = (i / len(full_text)) * 100
                print(f"📤 Stream ilerleme: {progress:.1f}% ({i}/{len(full_text)})")

            time.sleep(0.01)  # Smooth streaming

        stream_time = time.time() - stream_start

        # Tamamlandı sinyali
        yield f"data: {json.dumps({'text': '', 'done': True})}\n\n"

        print(f"\n✅ STREAMING TAMAMLANDI!")
        print(f"⏱️ Stream süresi: {stream_time:.2f} saniye")
        print(f"⏱️ Toplam süre: {(generation_time + stream_time):.2f} saniye")
        print("=" * 80 + "\n")

    except Exception as e:
        error_msg = f"❌ Üretim hatası: {str(e)}"
        print(error_msg)
        import traceback
        traceback.print_exc()
        yield f"data: {json.dumps({'error': error_msg, 'done': True})}\n\n"

    finally:
        # Bellek temizliği
        print("🧹 Bellek temizliği yapılıyor...")
        torch.cuda.empty_cache()
        gc.collect()
        print("✅ Bellek temizlendi")

# ============================================================================
# FLASK UYGULAMASI
# ============================================================================
app = Flask(__name__)

@app.route('/api/generate', methods=['POST'])
def generate():
    """Streaming proje analizi endpoint'i"""
    request_time = datetime.now()

    print("\n" + "=" * 80)
    print(f"📥 YENİ STREAMING İSTEK GELDİ")
    print(f"⏰ Zaman: {request_time.strftime('%H:%M:%S')}")
    print("=" * 80)

    try:
        # Veri doğrulama
        print("🔍 Veri doğrulanıyor...")
        data = request.get_json()

        if not data:
            print("❌ JSON girdisi bulunamadı!")
            return jsonify({"error": "JSON girdisi bulunamadı"}), 400

        print(f"✅ JSON verisi alındı - Boyut: {len(str(data))} byte")

        json_input = data.get("json_input")
        project_key = data.get("project_key", "PROJECT")

        if not json_input:
            print("❌ json_input alanı eksik!")
            return jsonify({"error": "json_input alanı gerekli"}), 400

        print(f"✅ Proje anahtarı: {project_key}")
        print(f"✅ Veri tipi: {type(json_input)}")

        # Eğer string ise parse et
        if isinstance(json_input, str):
            try:
                json_input = json.loads(json_input)
                print("✅ JSON string parse edildi")
            except:
                print("⚠️ JSON parse edilemedi, string olarak devam ediliyor")

        # Prompt oluştur
        prompt = create_analysis_prompt(json_input, project_key)

        if prompt.startswith("❌"):
            return jsonify({"error": prompt}), 400

        # Streaming response döndür
        print("🚀 Streaming response başlatılıyor...")
        return Response(
            stream_with_context(generate_streaming_response(prompt)),
            mimetype='text/event-stream',
            headers={
                'Cache-Control': 'no-cache',
                'X-Accel-Buffering': 'no',
                'Connection': 'keep-alive',
                'Access-Control-Allow-Origin': '*'
            }
        )

    except Exception as e:
        print(f"\n❌ HATA OLUŞTU!")
        print(f"Hata mesajı: {str(e)}")
        import traceback
        print("\n📋 Stack trace:")
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint'i"""
    print(f"💚 Health check - {datetime.now().strftime('%H:%M:%S')}")

    try:
        gpu_available = torch.cuda.is_available()
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3 if gpu_available else 0

        return jsonify({
            "status": "healthy",
            "model": "ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
            "gpu_available": gpu_available,
            "gpu_memory_gb": round(gpu_memory, 2),
            "timestamp": datetime.now().isoformat()
        }), 200
    except Exception as e:
        print(f"❌ Health check hatası: {e}")
        return jsonify({"status": "unhealthy", "error": str(e)}), 500

# ============================================================================
# TEST ENDPOINTİ (Debug için)
# ============================================================================
@app.route('/api/test', methods=['GET'])
def test():
    """Basit test endpoint'i"""
    print("🧪 Test endpoint çağrıldı")

    test_data = {
        "issues": [
            {"key": "TEST-1", "status": "Done", "priority": "High"},
            {"key": "TEST-2", "status": "In Progress", "priority": "Medium"},
            {"key": "TEST-3", "status": "To Do", "priority": "Low"}
        ]
    }

    prompt = create_analysis_prompt(test_data, "TEST")

    return Response(
        stream_with_context(generate_streaming_response(prompt)),
        mimetype='text/event-stream',
        headers={'Cache-Control': 'no-cache', 'X-Accel-Buffering': 'no'}
    )

# ============================================================================
# NGROK SETUP
# ============================================================================
try:
    from pyngrok import ngrok

    print("\n" + "=" * 80)
    print("🌐 NGROK BAŞLATILIYOR...")
    print("=" * 80)

    ngrok.kill()

    public_url = ngrok.connect(5000)
    print(f"\n✅ API HAZIR!")
    print(f"📡 Public URL: {public_url}")
    print(f"\n🔗 Endpoints:")
    print(f"   • Streaming: {public_url}/api/generate")
    print(f"   • Test: {public_url}/api/test")
    print(f"   • Health: {public_url}/health")

except ImportError:
    print("\n⚠️ pyngrok yüklü değil. Local olarak çalışılıyor.")
except Exception as e:
    print(f"❌ Ngrok hatası: {e}")

# ============================================================================
# SERVER BAŞLATMA
# ============================================================================
print("\n" + "=" * 80)
print("🚀 FLASK SERVER BAŞLATILIYOR...")
print("=" * 80)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, threaded=True, debug=False)

INFO 10-25 20:02:31 [__init__.py:216] Automatically detected platform cuda.
🚀 MODEL YÜKLEME BAŞLIYOR...
⏰ Zaman: 20:02:41
INFO 10-25 20:02:41 [utils.py:233] non-default args: {'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 4096, 'disable_log_stats': True, 'model': 'ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

INFO 10-25 20:02:59 [model.py:547] Resolved architecture: LlamaForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


...


🏁 Finish Reason: stop

✅ MODEL ÇIKTI ÜRETTİ!
📏 Çıktı uzunluğu: 2170 karakter
📏 Token sayısı: ~277
⏱️ Üretim süresi: 13.34 saniye
🚀 Hız: 162.7 karakter/saniye

📖 Önizleme:
**Proje Özeti:**

Verilen P987 projesinin toplamda 5 görevi bulunmaktadır. Görevlerin tamamında da "Bekleme" (Pending) olarak durumdadır. Bu nedenle proje genelinde %0 tamamlanma oranına sahip olduğun...
🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄

📡 STREAMING BAŞLIYOR...
🧹 Bellek temizliği yapılıyor...
✅ Bellek temizlendi

📥 YENİ STREAMING İSTEK GELDİ
⏰ Zaman: 20:56:47
🔍 Veri doğrulanıyor...
✅ JSON verisi alındı - Boyut: 647 byte
✅ Proje anahtarı: p987
✅ Veri tipi: <class 'dict'>
📝 Prompt oluşturuluyor...
✅ Chat template uygulandı
✅ Prompt hazır - Uzunluk: 2105 karakter
🚀 Streaming response başlatılıyor...

🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄
🤖 MODEL ÇALIŞIYOR...
⏰ Başlangıç: 20:56:47.383
📊 Prompt uzunluğu: 2105 karakter
🎚️ Temperature: 0.7
🎯 Max tokens: 2048
🛑 Stop tokens: []

📥 YENİ STREAMING İSTEK GELDİ
⏰ Zaman

INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 20:56:59] "POST /api/generate HTTP/1.1" 200 -



🏁 Finish Reason: stop

✅ MODEL ÇIKTI ÜRETTİ!
📏 Çıktı uzunluğu: 3092 karakter
📏 Token sayısı: ~385
⏱️ Üretim süresi: 9.79 saniye
🚀 Hız: 315.9 karakter/saniye

📖 Önizleme:
**PROJE RAPORU:**

**Proje Özeti:**
Projenin adı "Planlama Uygulaması" olup, planlama uygulamanızın tasarımı, geliştirilmesi, test edilmesi ve dağıtılmasını içermektedir. Şu anda toplamda 5 görev bulu...
🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄

📡 STREAMING BAŞLIYOR...
📤 Stream ilerleme: 6.5% (200/3092)
📤 Stream ilerleme: 12.9% (400/3092)
📤 Stream ilerleme: 19.4% (600/3092)
📤 Stream ilerleme: 25.9% (800/3092)
📤 Stream ilerleme: 32.3% (1000/3092)
📤 Stream ilerleme: 38.8% (1200/3092)
📤 Stream ilerleme: 45.3% (1400/3092)
📤 Stream ilerleme: 51.7% (1600/3092)
📤 Stream ilerleme: 58.2% (1800/3092)
📤 Stream ilerleme: 64.7% (2000/3092)
📤 Stream ilerleme: 71.2% (2200/3092)
📤 Stream ilerleme: 77.6% (2400/3092)
📤 Stream ilerleme: 84.1% (2600/3092)
📤 Stream ilerleme: 90.6% (2800/3092)
📤 Stream ilerleme: 97.0% (3000/3092)

✅ STREAM

INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 20:58:10] "POST /api/generate HTTP/1.1" 200 -



🏁 Finish Reason: stop

✅ MODEL ÇIKTI ÜRETTİ!
📏 Çıktı uzunluğu: 2683 karakter
📏 Token sayısı: ~318
⏱️ Üretim süresi: 129.33 saniye
🚀 Hız: 20.7 karakter/saniye

📖 Önizleme:
**Proje Özeti:**

p987 Projesinin özetini inceleyerek toplamda beş adet göreve sahip olduğunu görüyoruz. Görevlerin tamamına bakarak bu projede hala planlama uygulamasının oluşturma sürecinde olduğumu...
🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄🔄

📡 STREAMING BAŞLIYOR...
🧹 Bellek temizliği yapılıyor...
✅ Bellek temizlendi
